# Import Statements

In [1]:
# Import all the necessary functions here
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import pandas as pd
import dask.dataframe as dd

import numpy as np

import os
import tensorflow as tf
from tensorflow.keras.regularizers import L2
from tensorflow import keras
from tensorflow.keras import layers

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Importing and Prepping Data

In [2]:
# This gives google colab access to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Pathing to file directory
pwd_lying = "/content/gdrive/MyDrive/Body position prediction/lying data"
pwd_sitting = "/content/gdrive/MyDrive/Body position prediction/sitting data"
pwd_standing = "/content/gdrive/MyDrive/Body position prediction/standing data"
#!cd "$pwd" && ls

# Note: For you to connect to the shared folder, you first have to add a
# shortcut from the shared folder to your personal folder before you can path
# your way into it
data_files_lying = !ls gdrive/MyDrive/'Body position prediction'/'lying data'
data_files_sitting = !ls gdrive/MyDrive/'Body position prediction'/'sitting data'
data_files_standing = !ls gdrive/MyDrive/'Body position prediction'/'standing data'

In [4]:
from numpy.lib.shape_base import split
new_data_files_lying = []
new_data_files_sitting = []
new_data_files_standing = []

# I'll need to split the names up in the list since it's grouping 2 at a time
for i in data_files_lying:
  new_data_files_lying.extend(i.split())
for i in data_files_sitting:
  new_data_files_sitting.extend(i.split())
for i in data_files_standing:
  new_data_files_standing.extend(i.split())

new_data_files_standing

['Ainsley_Standing_30_Minutes_2.csv',
 'Bowen_Standing_30_Minutes.csv',
 'Ainsley_Standing_30_Minutes.csv',
 'Sarah_Standing_10_Minutes.csv',
 'Bowen_Standing_10_Minutes2.csv',
 'Sarah_Standing_11_Minutes.csv',
 'Bowen_Standing_10_Minutes.csv',
 'Sarah_Standing_20_Minutes_1.csv',
 'Bowen_Standing_15_Minutes_Walking.csv',
 'Sarah_Standing_20_Minutes_2.csv']

In [5]:
raw_lying_data = pd.DataFrame()
raw_sitting_data = pd.DataFrame()
raw_standing_data = pd.DataFrame()


for i in range(len(new_data_files_lying)):
  print(new_data_files_lying[i])
  df = pd.read_csv(pwd_lying + "/" + new_data_files_lying[i], header=1)
  raw_lying_data = pd.concat([raw_lying_data, df])

for i in range(len(new_data_files_sitting)):
  print(new_data_files_sitting[i])
  df = pd.read_csv(pwd_sitting + "/" + new_data_files_sitting[i], header=1)
  raw_sitting_data = pd.concat([raw_sitting_data, df])

for i in range(len(new_data_files_standing)):
  print(new_data_files_standing[i])
  df = pd.read_csv(pwd_standing + "/" + new_data_files_standing[i], header=1)
  raw_standing_data = pd.concat([raw_standing_data, df])


Ainsley_Lying_30_Minutes_2.csv
Bowen_Lying_20_Minutes.csv
Sarah_Lying_20_Minutes_3.csv
Ainsley_Lying_30_Minutes.csv
Bowen_Lying_21_Minutes.csv
Sarah_Lying_20_Minutes.csv
Bowen_Lying_20_Minutes_2.csv
Sarah_Lying_20_Minutes_2.csv
Ainsley_Sitting_30_Minutes_2.csv
Bowen_Sitting_11_Minutes_2.csv
Sarah_Sitting_20_Minutes_2.csv
Ainsley_Sitting_30_Minutes.csv
Bowen_Sitting_11_Minutes.csv
Sarah_Sitting_20_Minutes_3.csv
Bowen_Sitting_10_Minutes.csv
Bowen_Sitting_40_Minutes.csv
Sarah_Sitting_20_Minutes.csv
Ainsley_Standing_30_Minutes_2.csv
Bowen_Standing_30_Minutes.csv
Ainsley_Standing_30_Minutes.csv
Sarah_Standing_10_Minutes.csv
Bowen_Standing_10_Minutes2.csv
Sarah_Standing_11_Minutes.csv
Bowen_Standing_10_Minutes.csv
Sarah_Standing_20_Minutes_1.csv
Bowen_Standing_15_Minutes_Walking.csv
Sarah_Standing_20_Minutes_2.csv


In [6]:
# Now, I'm filtering the table since I only want time and the accelerometer data
ly = raw_lying_data[["timestamp", "green", "green2", "ir", "red", "acceleration_x", "acceleration_y", "acceleration_z"]]
sit = raw_sitting_data[["timestamp", "green", "green2", "ir", "red", "acceleration_x", "acceleration_y", "acceleration_z"]]
stand = raw_standing_data[["timestamp", "green", "green2", "ir", "red", "acceleration_x", "acceleration_y", "acceleration_z"]]

ly['body postion'] = 1
sit['body postion'] = 2
stand['body postion'] = 3

<ipython-input-6-31a3e4792fcd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ly['body postion'] = 1
<ipython-input-6-31a3e4792fcd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sit['body postion'] = 2
<ipython-input-6-31a3e4792fcd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [7]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()

ppg_sitting_values = sit[["green", "green2", "ir", "red"]].values
ppg_lying_values = ly[["green", "green2", "ir", "red"]].values
ppg_standing_values = stand[["green", "green2", "ir", "red"]].values #returns a numpy array
accel_sitting_values = sit[["acceleration_x", "acceleration_y", "acceleration_z"]].values
accel_standing_values = stand[["acceleration_x", "acceleration_y", "acceleration_z"]].values
accel_lying_values = ly[["acceleration_x", "acceleration_y", "acceleration_z"]].values


sitting_values_norm = min_max_scaler.fit_transform(ppg_sitting_values)
lying_values_norm = min_max_scaler.fit_transform(ppg_lying_values)
standing_values_norm = min_max_scaler.fit_transform(ppg_standing_values)

sitting_values = np.concatenate((sitting_values_norm, accel_sitting_values), axis=1)
standing_values = np.concatenate((standing_values_norm, accel_standing_values), axis=1)
lying_values = np.concatenate((lying_values_norm, accel_lying_values), axis=1)


sitting = pd.DataFrame(sitting_values)
lying = pd.DataFrame(lying_values)
standing = pd.DataFrame(standing_values)

In [8]:
ppg_sitting_values

array([[    66,     65, 332306,   1038],
       [   223,    224, 429772,   3778],
       [   112,    111, 367795,   1366],
       ...,
       [ 58398,  58414, 392307, 378687],
       [ 58302,  58305, 392294, 378681],
       [ 58204,  58205, 392297, 378716]])

In [9]:
sitting

,0,1,2,3,4,5,6
0,0.000000,0.000000,0.446089,0.000485,0.609,-0.139,0.681
1,0.000299,0.000303,0.727302,0.005719,0.732,-0.134,0.544
2,0.000088,0.000088,0.548484,0.001112,0.445,-0.228,0.542
3,0.000141,0.000139,0.552555,0.000913,0.643,-0.119,0.654
4,0.000233,0.000233,0.537935,0.000741,0.650,-0.109,0.664
...,...,...,...,...,...,...,...
283770,0.110934,0.110940,0.618846,0.721669,0.368,-0.861,4.524
283771,0.111056,0.111088,0.618987,0.721749,0.371,-0.793,4.537
283772,0.111274,0.111306,0.619206,0.721874,0.356,-0.794,4.541
283773,0.111091,0.111098,0.619169,0.721862,0.368,-0.842,4.545


In [10]:
# Put it in a tensor

tt_lying = torch.tensor(lying[[0,1,2,3,4,5,6]].values)
tt_sitting = torch.tensor(sitting[[0,1,2,3,4,5,6]].values)
tt_standing = torch.tensor(standing[[0,1,2,3,4,5,6]].values)

In [11]:
unfolded_tt_lying = tt_lying.unfold(0, 1000, 50)
unfolded_tt_sitting = tt_sitting.unfold(0, 1000, 50)
unfolded_tt_standing = tt_standing.unfold(0, 1000, 50)

# Creating Training and Testing Datasets

In [12]:
train_tt_accels = torch.empty(0, 7, 1000)
test_tt_accels = torch.empty(0, 7, 1000)

train_label = []
test_label = []

unfolded_tt_accels = [unfolded_tt_lying, unfolded_tt_sitting, unfolded_tt_standing]

for i in range(len(unfolded_tt_accels)):
  train_dataset, test_dataset = torch.utils.data.random_split(unfolded_tt_accels[i], [0.8,.2])

  train = torch.stack([t for t in train_dataset])
  test = torch.stack([t for t in test_dataset])

  train_tt_accels = torch.cat((train_tt_accels, train), 0)
  test_tt_accels = torch.cat((test_tt_accels, test), 0)

  for j in range(len(train)):
    train_label.append(i)
  for j in range(len(test)):
    test_label.append(i)

train_tt_accels.shape, test_tt_accels.shape

(torch.Size([12776, 7, 1000]), torch.Size([3193, 7, 1000]))

# ML Model

In [13]:
def build_model():
    model = Sequential()
    model.add(layers.LSTM(32, activation='relu', return_sequences=True, input_shape=(7, 1000)))
    model.add(layers.LSTM(32, activation='relu'))

    model.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dropout(0.3))
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dropout(0.3))
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(3, activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


# Define your training and validation data
x_train = train_tt_accels.numpy()
y_train = np.array(train_label)
x_val = test_tt_accels.numpy()
y_val = np.array(test_label)

x_train.shape

# Build the model
model = build_model()

# Set up ModelCheckpoint callback to save the best model
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model with callbacks
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=[checkpoint])



Epoch 1/50
398/400 [============================>.] - ETA: 0s - loss: 0.9313 - accuracy: 0.6192
Epoch 1: val_accuracy improved from -inf to 0.81365, saving model to best_model.h5
400/400 [==============================] - 13s 21ms/step - loss: 0.9301 - accuracy: 0.6197 - val_loss: 0.5369 - val_accuracy: 0.8137
Epoch 2/50
  3/400 [..............................] - ETA: 11s - loss: 0.7150 - accuracy: 0.7292

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - ETA: 0s - loss: 0.5672 - accuracy: 0.8051
Epoch 2: val_accuracy improved from 0.81365 to 0.85092, saving model to best_model.h5
400/400 [==============================] - 9s 23ms/step - loss: 0.5672 - accuracy: 0.8051 - val_loss: 0.4298 - val_accuracy: 0.8509
Epoch 3/50
399/400 [============================>.] - ETA: 0s - loss: 0.4767 - accuracy: 0.8409
Epoch 3: val_accuracy improved from 0.85092 to 0.87159, saving model to best_model.h5
400/400 [==============================] - 9s 24ms/step - loss: 0.4766 - accuracy: 0.8409 - val_loss: 0.3555 - val_accuracy: 0.8716
Epoch 4/50
399/400 [============================>.] - ETA: 0s - loss: 0.4339 - accuracy: 0.8521
Epoch 4: val_accuracy improved from 0.87159 to 0.88162, saving model to best_model.h5
400/400 [==============================] - 8s 19ms/step - loss: 0.4339 - accuracy: 0.8521 - val_loss: 0.3544 - val_accuracy: 0.8816
Epoch 5/50
397/400 [============================>.] - ETA: 0s - loss:

KeyboardInterrupt: 

In [14]:
# Evaluate the best model
best_model = build_model()  # Rebuild the model
best_model.load_weights("best_model.h5")  # Load the best weights
loss, accuracy = best_model.evaluate(x_val, y_val)
print(f"Validation loss of the best model: {loss}, Validation accuracy of the best model: {accuracy}")

100/100 [==============================] - 2s 8ms/step - loss: 0.1993 - accuracy: 0.9251
Validation loss of the best model: 0.199269101023674, Validation accuracy of the best model: 0.9251487851142883


In [15]:
model.save('my_model.keras')